## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
import pandas as pd

In [4]:
def my_iterrows():
    suma = 0
    counter = 0
    for index, row in reviews.iterrows():
        if '2011-01-01' > row.date >= '20010-01-01':
            counter += 1
            suma += row['rating']
    return suma / counter

def my_iterrows_2010():
    suma = 0
    for index, row in reviews_2010.iterrows():
        suma += row['rating']
    return suma / reviews_2010.shape[0]

def Series_mean():
    return reviews_2010.mean()

reviews = pd.read_csv('reviews_visualization.csv', delimiter=',', index_col=0)
reviews_2010 = reviews[(reviews.date >= '20010-01-01') & (reviews.date <'2011-01-01')]

In [7]:
%time
print(my_iterrows())

CPU times: total: 0 ns
Wall time: 0 ns
4.5230757650702165


In [8]:
%time
print(my_iterrows_2010())

CPU times: total: 0 ns
Wall time: 940 µs
4.5230757650702165


In [ ]:
%time
print(Series_mean())

CPU times: total: 0 ns
Wall time: 0 ns


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [22]:
!pip install line_profiler
%load_ext line_profiler
def my_iterrows_2010():
    suma = 0
    for index, row in reviews_2010.iterrows():
        suma += row['rating']
    return suma / reviews_2010.shape[0]

print(my_iterrows_2010())
%lprun -f my_iterrows_2010 my_iterrows_2010()

Defaulting to user installation because normal site-packages is not writeable
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
4.5230757650702165


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [28]:
!pip install line_profiler
%load_ext line_profiler
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

reviews = pd.read_csv('reviews_visualization.csv', delimiter=',', index_col=0)
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Defaulting to user installation because normal site-packages is not writeable
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [51]:
def MAPE1(recipe):
    absolute_deviation = 0
    reviews_recipe = reviews[reviews['recipe_id']==recipe]
    n = len(reviews_recipe)
    mean_rating = reviews_recipe.rating.sum() / n
    for name, value in reviews_recipe[['rating']].iteritems():
        absolute_deviation += abs(value - mean_rating)
    mape = absolute_deviation/n/mean_rating*100
    return mape

reviews = pd.read_csv('reviews_visualization.csv', delimiter=',', index_col=0)
print(MAPE1(21752))

960803    0.403226
960847    3.629032
960848    3.629032
960801    9.274194
960799    3.629032
960800    3.629032
960797    9.274194
960798    3.629032
Name: rating, dtype: float64


In [64]:
import numba
from numba import jit, njit
@jit(nopython=True)
def MAPE2(recipe):
    absolute_deviation = 0
    reviews_recipe = reviews[reviews['recipe_id']==recipe]
    n = len(reviews_recipe)
    mean_rating = reviews_recipe.rating.sum() / n
    for name, value in reviews_recipe[['rating']].iteritems():
        absolute_deviation += abs(value - mean_rating)
    mape = absolute_deviation/n/mean_rating*100
    return mape
reviews = pd.read_csv('reviews_visualization.csv', delimiter=',', index_col=0)
reviews = reviews.to_numpy()
print(MAPE2(21752))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(readonly array(pyobject, 2d, F), Literal[str](recipe_id))
 
There are 22 candidate implementations:
[1m  - Of which 20 did not match due to:
  Overload of function 'getitem': File: <numerous>: Line N/A.
    With argument(s): '(readonly array(pyobject, 2d, F), unicode_type)':[0m
[1m   No match.[0m
[1m  - Of which 1 did not match due to:
  Overload in function 'GetItemBuffer.generic': File: numba\core\typing\arraydecl.py: Line 166.
    With argument(s): '(readonly array(pyobject, 2d, F), unicode_type)':[0m
[1m   Rejected as the implementation raised a specific error:
     NumbaTypeError: [1munsupported array index type unicode_type in [unicode_type][0m[0m
  raised from C:\anaconda3\lib\site-packages\numba\core\typing\arraydecl.py:72
[1m  - Of which 1 did not match due to:
  Overload in functi

In [ ]:
import numpy as np
from numba import jit

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'